# _Promise_

## new Promise()

### The constructor `Promise()` must be used with `new`, and must be provided a function callback
### This function is passed two function callbacks that act as resolution capabilities for the promise, commonly denoted as `resolve()` and `reject()`

In [1]:
new Promise((resolve, reject) => {
    //...
});

[Promise] {}

### `reject()` simply rejects the promise

In [2]:
{

const p = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise is rejected.


### `resolve()` can either fulfill or reject the promise
- **If `resolve()` is passed a non-_Promise_ value, then the promise is fulfilled with that value**

- **If `resolve()` is passed a _Promise_, then the promise will adopt the resolution of the passed promise**

## _Promise_ is immutable

### Any `then()` registered callbacks will only ever be called once (each)
### If `resolve()` or `reject()` is called multiple times, or tries to call both, the _Promise_ will accept only the first resolution, and silently ignore any subsequent attempts

In [20]:
{

const p = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
    setTimeout(reject, 0);
    setTimeout(resolve, 0);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise rejected.


### If an exception is thrown at any point in the creation of a _Promise_, or in the observation of its resolution, such as a `TypeError` or `ReferenceError`, that exception will be caught, and it will force the _Promise_ to reject

In [ ]:
{
/* RUN IN BABEL KERNEL */
const p = new Promise(resolve => {
    let str = '';
    str.push(0);
    resolve(str);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    (e) => {
        console.log(`Promise rejected. ${e}.`);
    }
);

}

### If you call `resolve()` or `reject()` with multiple parameters, all subsequent parameters beyond the first will be silently ignored

In [4]:
{

const p = new Promise(resolve => {
    resolve('a', 'b');
})
.then((arg0, arg1) => {
    console.log(`arg0: '${arg0}'`);
    console.log(`arg1: ${arg1}`);
});

}

arg0: 'a'
arg1: undefined


### `Promise.prototype.then()` returns a new _Promise_

In [5]:
{

const promise = new Promise(resolve => {
    resolve();
});

const thenPromise = promise.then(() => {});

console.log(thenPromise.toString());

}

[object Promise]


## `Promise.resolve()`

### Returns a _Promise_ that's fulfilled with that value

In [6]:
{

const p = Promise.resolve(42);

p.then(value => {
    console.log(value);
});

}

[Promise] {}

42


### If you pass a _Promise_ to `Promise.resolve()`, you just get the same promise back

In [7]:
{

const p1 = new Promise(resolve => {
    resolve(42);
});

const p2 = Promise.resolve(p1).then(value => {
    console.log(value);
});

}

42


### Inside the fulfillment/rejection handlers, if you return a value, the new returned _Promise_ is resolved

In [9]:
{

const p = new Promise(resolve => {
    resolve();
})
.then(() => {
    return 42;
})
.then(value => {
    console.log(value)
});

}

42


### Inside the fulfillment/rejection handlers, if an exception is thrown, the new returned _Promise_ is resolved

In [11]:
{

const p = new Promise(resolve => {
    resolve();
})
.then(() => {
    var str = '';
    str.push(0);
    return str;
})
.then(
    () => {
        console.log('Promise fulfilled.')
    },
    (e) => {
        console.log(`Promise rejected: ${e}.`)
    }
);

}

Promise rejected: TypeError: str.push is not a function.


### If the fulfillment or rejection handlers returns a _Promise_, it is unwrapped, so that whatever its resolution is will become the resolution of the chained _Promise_ returned from the current `then()`

In [12]:
{

const p = new Promise(resolve => {
    resolve();
})
.then(() => {
    return new Promise((resolve, reject) => {
        setTimeout(() => reject(), 0);
    });
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise rejected.


## `Promise.all()`

### `Promise.all()` expects a single argument: an array of _Promise_ instances. The _Promise_ returned from `Promise.all()` will receive a fulfillment argument that is an array of all the fulfillment values from the passed in _Promises_, in the same order as specified

In [13]:
{

const p1 = new Promise(resolve => {
    resolve(1);
});

const p2 = new Promise(resolve => {
    resolve(2);
});

Promise.all([p1, p2])
.then(values => console.log(values));

}

[Promise] {}

[Array] [1,2]


### If any one of those _Promise_ instances rejects, the main `Promise.all()` _Promise_ is immediately rejected, discarding all results from any other _Promise_ instances

In [14]:
{

const p1 = new Promise(resolve => {
    resolve();
});

const p2 = new Promise((resolve, reject) => {
    setTimeout(() => reject('Error 42'), 0);
});

Promise.all([p1, p2])
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    (e) => {
        console.log(`Promise rejected: ${e}.`);
    }
);

}

[Promise] {}

Promise rejected: Error 42.


## `Promise.race()`

### `Promise.race()` expects a single array argument containing one or more _Promise_ instance
### `Promise.race()` will fulfill if and when any _Promise_ resolution is a fulfillment

In [15]:
{

const p1 = new Promise(resolve => {
    setTimeout(() => resolve('p1'), 1000);
});

const p2 = new Promise(resolve => {
    resolve('p2');
});

Promise.race([p1, p2])
.then((value) => {
    console.log(`The first Promise to resolve: ${value}.`);
});

}

[Promise] {}

The first Promise to resolve: p2.


### `Promise.race()` will reject if and when any _Promise_ resolution is a rejection

In [16]:
{

const p1 = new Promise(resolve => {
    setTimeout(() => resolve('p1'), 1000);
});

const p2 = new Promise((resolve, reject) => {
    setTimeout(() => reject('p2'), 0);
});

Promise.race([p1, p2])
.then(
    () => {},
    (e) => {
        console.log(`The first Promise to resolve: ${e}.`)
    }
)

}

[Promise] {}

The first Promise to resolve: p2.


### If `Promise.race()` is passed an empty array, it will never resolve

In [17]:
{

Promise.race([])
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

[Promise] {}

In [19]:
{

function calculateAge(birthDate, delay) {
    return new Promise(resolve => {
        setTimeout(() => {
            let age = ((Date.now() - birthDate.getTime()) / 1000 / 60 / 60 / 24 / 365).toFixed();
            resolve(age);
        }, delay);
    });
}

function* sayMyAge(name, birthDate, delay) {
    let age = yield calculateAge(birthDate, delay);
    console.log(`${name} is ${age} years old!`);
}

const it1 = sayMyAge('Doug', new Date(1991, 2, 9), 1000);
it1.next().value.then(age => {
    it1.next(age);
});

const it2 = sayMyAge('Philippe', new Date(1991, 9, 22), 10);
it2.next().value.then(age => {
    it2.next(age);
});

}

[Promise] {}

Philippe is 25 years old!
Doug is 26 years old!
